In [27]:
!pip install transformers
!pip install pytorchvideo transformers
!pip install av

     ---------------------------------------- 0.0/132.7 kB ? eta -:--:--
     -------------------------------------- 132.7/132.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.2 kB ? eta -:--:--
     ---------------------------------------- 50.2/50.2 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/42.2 kB ? eta -:--:--
     ---------------------------------------- 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none

In [6]:
import av

In [2]:
from transformers import VideoMAEImageProcessor, VideoMAEForPreTraining
import numpy as np
import torch

num_frames = 16
video = list(np.random.randn(16, 3, 224, 224))

processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForPreTraining.from_pretrained("MCG-NJU/videomae-base")

pixel_values = processor(video, return_tensors="pt").pixel_values

num_patches_per_frame = (model.config.image_size // model.config.patch_size) ** 2
seq_length = (num_frames // model.config.tubelet_size) * num_patches_per_frame
bool_masked_pos = torch.randint(0, 2, (1, seq_length)).bool()

outputs = model(pixel_values, bool_masked_pos=bool_masked_pos)
loss = outputs.loss

ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-4.538973870330655, 4.226492976822624] which cannot be converted to uint8.

In [3]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(

    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"

)

In [23]:

def normalize(x):
    min_x = np.min(x, axis=0)
    max_x = np.max(x, axis=0)
    return (x - min_x) / (max_x - min_x)

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    print()
    return np.stack([normalize(x.to_ndarray(format="rgb24")) for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [24]:
container = av.open(file_path)

# sample 16 frames

indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)

In [25]:
from transformers import AutoImageProcessor, VideoMAEModel
from huggingface_hub import hf_hub_download
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")

model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [26]:
# prepare video for the model
inputs = image_processor(list(video), return_tensors="pt")
# forward pass
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-4.538973870330655, 4.226492976822624] which cannot be converted to uint8.

In [ ]:
video